1.  function UNIFY(x , y, θ) returns a substitution to make x and y identical

    inputs: x, a string storing a variable, constant, list, or compound expression
    y, a string storing a a variable, constant, list, or compound expression
    θ, the substitution built up so far (optional, defaults to empty)

    if θ = failure then return failure
    else if x = y then return θ
    else if VARIABLE(x ) then
        return UNIFY-VAR(x , y, θ)
    else if VARIABLE(y) then
        return UNIFY-VAR(y, x , θ)
    else if COMPOUND(x ) and COMPOUND(y) then
        return UNIFY(x.ARGS, y.ARGS, UNIFY(x.OP, y.OP, θ))
    else if LIST(x ) and LIST(y) then
        return UNIFY(x .REST, y.REST, UNIFY(x .FIRST, y.FIRST, θ))
    else return failure

2.  function UNIFY-VAR(var, x , θ) returns a substitution
    if {var/val} ∈ θ then
        return UNIFY(val , x , θ)
    else if {x/val} ∈ θ then
        return UNIFY(var, val , θ)
    else if OCCUR-CHECK(var, x ) then
        return failure
    else return add {var/x } to θ

In [1]:
import re

def is_function(sentence):
    match = re.match(r'(\w+)\((.*)\)', sentence)
    if match:
        function_name = match.group(1)
        parameters = match.group(2).split(',')
        return function_name, [param.strip() for param in parameters]
    return None

def replace_with_theta(value, theta):
    while value in theta:
        value = theta[value]
    return value

def unify_parameters(param1, param2, theta={}):
    param1 = replace_with_theta(param1, theta)
    param2 = replace_with_theta(param2, theta)

    if param1 == param2:
        return theta
    elif is_variable(param1) and (not is_variable(param2) or is_function(param2)):
        return unify_var(param1, param2, theta)
    elif is_variable(param2) and (not is_variable(param1) or is_function(param1)):
        return unify_var(param2, param1, theta)
    elif is_function(param1) and is_function(param2):
        name1, params1 = is_function(param1)
        name2, params2 = is_function(param2)

        if name1 != name2 or len(params1) != len(params2):
            return "failure"

        for p1, p2 in zip(params1, params2):
            result = unify_parameters(p1, p2, theta)
            if result == "failure":
                return "failure"

        for i in range(len(params1)):
            if is_function(params1[i]) and is_function(params2[i]):
                inner_theta = unify_parameters(params1[i], params2[i], theta)
                if inner_theta == "failure":
                    return "failure"
                theta.update(inner_theta)


                for j in range(len(params1)):
                    params1[j] = replace_with_theta(params1[j], theta)
                    params2[j] = replace_with_theta(params2[j], theta)

        return theta

    return "failure"

def unify_var(var, value, theta):
    if var in theta:
        return unify_parameters(theta[var], value, theta)
    elif value in theta:
        return unify_parameters(var, theta[value], theta)
    else:
        theta[var] = value
        print(f"{var}/{value}")
        return theta

def is_variable(x):
    return type(x) == str and len(x) == 1 and (x.islower() or x.isupper())

def unify_sentences(sentence1, sentence2):

    func1 = is_function(sentence1)
    func2 = is_function(sentence2)

    if func1 and func2:
        name1, params1 = func1
        name2, params2 = func2

        if name1 != name2 or len(params1) != len(params2):
            return "failure"

        theta = {}
        for p1, p2 in zip(params1, params2):
            result = unify_parameters(p1, p2, theta)
            if result == "failure":
                return "failure"

        for key in list(theta.keys()):
            theta[key] = replace_with_theta(theta[key], theta)

        return theta
    return "failure"

sentence1 = "Prime(13)"
sentence2 = "Prime(y)"

sentence3 = "Knows(John, x)"
sentence4 = "Knows(y, Mother(y))"

sentence5 = "pro(b(x), X, f(g(Z)))"
sentence6 = "pro(Z, f(Y), f(Y))"

sentence7 = "Quick(a, g(x, a), f(y))"
sentence8 = "Quick(a, g(f(b), a), x)"

print("Unifying Prime(13) and Prime(y):")
result1 = unify_sentences(sentence1, sentence2)
print(result1)

print("\nUnifying Knows(John, x) and Knows(y, Mother(y)):")
result2 = unify_sentences(sentence3, sentence4)
print(result2)

print("\nUnifying pro(b(x), X, f(g(Z))) and pro(Z, f(Y), f(Y)):")
result3 = unify_sentences(sentence5, sentence6)
print(result3)

print("\nUnifying Quick(a, g(x, a), f(y)) and Quick(a, g(f(b), a), x):")
result4 = unify_sentences(sentence7, sentence8)
print(result4)


Unifying Prime(13) and Prime(y):
y/13
{'y': '13'}

Unifying Knows(John, x) and Knows(y, Mother(y)):
y/John
x/Mother(y)
{'y': 'John', 'x': 'Mother(y)'}

Unifying pro(b(x), X, f(g(Z))) and pro(Z, f(Y), f(Y)):
Z/b(x)
X/f(Y)
Y/g(Z)
{'Z': 'b(x)', 'X': 'f(Y)', 'Y': 'g(Z)'}

Unifying Quick(a, g(x, a), f(y)) and Quick(a, g(f(b), a), x):
failure
